In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import importlib
import training as training
import connect4cnn as connect4cnn
importlib.reload(training);
importlib.reload(connect4cnn)
from connect4cnn import Connect4Cnn


In [66]:
#
# Hyper parameter
#
gamma = 0.9 # Q-learning's discount factor, should probably stay constant at 0.9
learning_rate = 0.1 # initial learning rate
actual_learning_rate = 0.1 # used to reset optimizers from file
epsilon = 0.01
validation_games = 5000
omega = 1 # the percentage of opponent random moves during validation


In [69]:

#
# Create the model and optimizer
# 
games = 0
model = Connect4Cnn()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
sum(p.numel() for p in model.parameters() if p.requires_grad)

34439

In [ ]:
#
# Load model from checkpoint
#
games = 1000000
training.loadCheckpoint(model, optimizer, f'connect4-{games}', actual_learning_rate)

In [ ]:
#
# TRAINING
#
gamesToGo = 500000
training.train(model, optimizer, gamesToGo, epsilon, omega, gameOffset = games, gamma = gamma)
games += gamesToGo

### Connect4 CNN training 
optimizer: SGD initial lr: 0.1  
validation 10000 games against normalized softmax moves and &omega; random moves  
exploration via normalized softmax and &epsilon;  
loss over last 50.000 games

| Games     | lr        | &epsilon; | Loss                  | &omega;   |Cross      | Circle    | Remarks
| :-------: | :-:       | :---:     | :-----------:         | :----:    |:----:     | :-----:   | :-------:
|           | 0.1       | 0.01      |                       | 1         | 84.32%    | 76.64%    |
| 50.000    |           |           |                       |           |           |           |
| 100.000   |           |           |                       |           |           |           |
| 150.000   |           |           |                       |           |           |           |
| 200.000   |           |           |                       |           |           |           |
| 250.000   |           |           |                       |           |           |           |
| 300.000   |           |           |                       |           |           |           |
| 350.000   |           |           |                       |           |           |           |
| 400.000   |           |           |                       |           |           |           |
| 450.000   |           |           |                       |           |           |           |
| 500.000   |           |           |                       |           |           |           |
| 550.000   |           |           |                       |           |           |           |
| 600.000   |           |           |                       |           |           |           |
| 650.000   |           |           |                       |           |           |           |
| 700.000   |           |           |                       |           |           |           |
| 750.000   |           |           |                       |           |           |           |
| 800.000   |           |           |                       |           |           |           |
| 850.000   |           |           |                       |           |           |           |
| 900.000   |           |           |                       |           |           |           |
| 950.000   |           |           |                       |           |           |           |
| 1.000.000 |           |           |                       |           |           |           |
